### Multi agents in Matrix-Games

One example of multi-agents Reinforcement Learning, important because of its connections to Game Theory, is that of Agents interacting in Matrix Games.

A Matrix Game is, for each player, similar to a multi-armed bandit in the sense that:

- There is just one state.
- There are multiple choices of actions.

What is different is that the reward (*payoff*) for a given action depends also on the actions of the other player. In fact it is what is called an Adversarial bandits problem.


#### $2$-players Matrix Games

Let us consider $2$-players Matrix Games with 2 actions. At each time $t$ the players simultaneously choose their actions $a_1, a_2$. 
The payoffs depend on both actions, and can be written as matrices; let us call $A$ the matrix of payoffs for player $1$ and $B$ the matrix of payoffs for player $2$. 


For player $1$:
$$
A = 
\begin{bmatrix} 
A_{1 1} & A_{1 2} \\
A_{2 1} & A_{2 2}\\
\end{bmatrix} 
$$

For player $2$:
$$
B = 
\begin{bmatrix} 
B_{1 1} & B_{1 2} \\
B_{2 1} & B_{2 2}\\
\end{bmatrix} 
$$

This means that if player $1$ acted with action $i$ and player $2$ acted with action $j$, they respective payoffs are $A_{i j}$ and $B_{i j}$. A conventional, condensed way to write them is:

$$
R = \begin{bmatrix} 
A_{1 1}, B_{1 1} & A_{1 2}, B_{1 2} \\
A_{2 1}, B_{2 1} & A_{2 2}, B_{2 2}\\
\end{bmatrix} 
$$


On average, given the policies 
- $\pi^{(1)}(1)$,    $\pi^{(1)}(2) = 1-\pi^{(1)}(1)$ 
- $\pi^{(2)}(1)$,    $\pi^{(2)}(2) = 1-\pi^{(2)}(1)$ 

the expected payoffs are:

$$
G^{(1)} = \sum_{i j} A_{i j} \pi^{(1)}(i) \pi^{(2)}(j)\\
G^{(2)} = \sum_{i j} B_{i j} \pi^{(1)}(i) \pi^{(2)}(j) 
$$

These expected payoffs are those we want to maximize.
Clearly, now there is no simple answer to the question of what are optimal policies: They depend on what the other agent does! 

Some games are trivial, in the sense that both agents "*want the same thing*". However the most interesting games are those where the two agents have conflicting aims. An important class of games is that of zero-sum reward games: $A_{i j} + B_{ij} = 0$.

Consider for example the case of the game of "matching pennies".  For player $1$:
$$
A = 
\begin{bmatrix} 
1 & -1 \\
-1 & 1\\
\end{bmatrix} 
$$

For player $2$:
$$
B = 
\begin{bmatrix} 
-1 & 1 \\
1 & -1\\
\end{bmatrix} 
$$

Player $1$ wants to play *the same action* as player 2, who however wants to play *the opposite action* as player $1$. What can the agents do? How can they try and adapt their strategies w.r.t. their opponent?

### RL with Natural Gradients and Matrix Games

One possible way to approach the problem is to construct *two* Reinforcement Learning agents, which will play against each other for many and many repetitions, and update their policies using a Policy Gradient method.
 
In particular, there is an important special choice of method, which is the one using the Natural Gradient and soft-max policies.

Recall that in this case we gave for each action a parameter $\theta$ such that:
$$
\pi_{\theta}(a) = \frac{e^{\theta_{a}}}{\sum_{a'} e^{\theta_{a'}}}
$$

It turns out (see Lecture 18) that the Natural Gradient for a Matrix Games, we have that the *gradient of the expected payoff* w.r.t. the parameters $\theta$ is:

$$
\tilde{\nabla}_{\theta_a^{(1)}}G^{(1)} = \sum_{a_2} A_{a \, a_2} \pi^{(2)}(a_2)
$$

What is the righten sideof the equation?
It is exactly the payoff that agent gets on average when it plays action $a$, and player $2$ plays its strategy $\pi^{(2)}(1), \pi^{(2)}(2)$. 


This means that at each time the action with higher payoff gets exponentially ($\sim e^{\theta_a^{(1)}}$!) more probable. (But since also the strategy of player $2$ changes, that same action can become less favorable in the future...).

The Natural Gradient algorithm for Adversarial Bandits is called also EXP3 *(EXPonential-weight algorithm for EXPloration and EXPloitation)*, and as such has been well studied for its properties of convergence.

In [11]:
import matplotlib.pyplot as plt
import numpy as np

class ActorCritic_NaturalGradient():
    def __init__(self, 
                 space_size, 
                 action_size, 
                 gamma=1, 
                 lr_v = 0.01,
                 lr_a = 0.01,
                 eta = 0.01,
                 start_prob = np.array([])):
        """
        Calculates optimal policy using in-policy Temporal Difference control
        Approximate V-value for states S!
        """        
        # the discount factor
        self.gamma = gamma
        # size of system
        self.space_size = space_size
        # action size
        self.action_size = action_size
        
        # the learning rate for value
        self.lr_v = lr_v
        # the learning rate for actor
        self.lr_a = lr_a
        
        # to diminuish variance
        self.eta = 0.01
        
        # Stores the Value Approximation weights
        self.w = np.zeros((*self.space_size,))
        # Stores the Policy parametrization
        self.Theta = np.zeros( (*self.space_size, self.action_size) )
    
        if not start_prob.size == 0:
            assert start_prob.shape == (*self.space_size, self.action_size), 'Starting policy does not match game dimensions.'
            self.Theta = np.log(start_prob)
    
    # -------------------   
    def single_step_update(self, s, a, r, new_s, done):
        """
        Uses a single step to update the values, using Temporal Difference for V values.
        Employs the EXPERIENCED action in the new state  <- Q(S_new, A_new).
        """
        
        # ---------------------
        # CRITIC UPDATE -------
        # ---------------------
        
        if done:
            # -----------------------
            delta = (r + 0 - self.w[(*s,)])
            
        else:
            # --------------------------
            delta = (r + 
                      self.gamma * self.w[(*new_s,)]
                                 - self.w[(*s,)])
            
        # --------------------
        self.w[(*s,)] += self.lr_v * delta 
        
        # -----------------------------------------------
        # Now Actor update with Natural Gradient --------
        # -----------------------------------------------
        policy = self.get_policy(s)
        
        # Natural Gradient: non-zero only for the action which has been selected!
        self.Theta[(*s, a) ] += self.lr_a * delta / (policy[a] + 0.001)

        
    # ---------------------
    def get_action(self, s):
        """
        Chooses action at random using policy from current parameters Theta.
        """
        # Prob(a|s) = exp(theta_a) / (sum_a' exp(theta_a') 

        # Common shift has no influence on relative probabilities
        # But very helpful: Otherwise it can explode! 
        log_prob = self.Theta[(*s,)] - np.mean(self.Theta[(*s,)])
        
        # Clip needed to avoid NaN.
        log_prob = np.clip(log_prob, -20, 20)
        
        # P(a_i) = exp(theta_i) / sum_j exp(theta_j) 
        prob_actions = np.exp(log_prob)
        prob_actions = prob_actions / np.sum(prob_actions)
        
        # take one action from the array of actions with the probabilities as defined above.
        a = np.random.choice(self.action_size, p=prob_actions)
        return a 
    
        # ---------------------
    def get_policy(self, s):
        """
        Returns policy from current parameters Theta.
        """
        # Prob(a|s) = exp(theta_a) / (sum_a' exp(theta_a') 
        
        # Common shift has no influence on relative probabilities
        # But very helpful: Otherwise it can explode! 
        log_prob = self.Theta[(*s,)] - np.mean(self.Theta[(*s,)])
        
        # Clip needed to avoid NaN.
        log_prob = np.clip(log_prob, -20, 20)
        
        prob_actions = np.exp(log_prob)
        prob_actions = prob_actions / np.sum(prob_actions)
            
        return prob_actions

### Evolution of policies


In [12]:
# ActorCritic with Natural Gradient: Bootstrapping to solve CartPole (with state aggregation).

n_episodes = 100000

gamma = 1

# Initialize 
lr_v_0 = 0.1
lr_a_0 = 0.0001

lr_v = lr_v_0
lr_a = lr_a_0

PayoffMatrix1 = np.array([[1, -1],
                          [-1, 1]]
                        )

PayoffMatrix2 = np.array([[-1, 1],
                          [1, -1]]
                        )

starting_prob_1 = np.array([[0.15, 0.85]])
starting_prob_2 = np.array([[0.85, 0.15]])

# Initialize algorithm
Player1 = ActorCritic_NaturalGradient(space_size=[1], 
                      action_size=PayoffMatrix1.shape[0],
                      gamma=gamma, 
                      lr_v=lr_v,
                      lr_a=lr_a,
                      start_prob=starting_prob_1)

# Initialize algorithm
Player2 = ActorCritic_NaturalGradient(space_size=[1], 
                      action_size=PayoffMatrix2.shape[0],
                      gamma=gamma, 
                      lr_v=lr_v,
                      lr_a=lr_a,
                      start_prob=starting_prob_2)

traj_policy_1 = np.empty([0])
traj_policy_2 = np.empty([0])

av_rew = np.zeros(2)
# RUN OVER EPISODES
done = False
for i in range(n_episodes):
    
    if (i%10000==0):
        print(i, end=' ')

    # There is only one state!
    s = [0]

    # Select action with Soft-max
    a_1 = Player1.get_action(s)
    a_2 = Player2.get_action(s)

    r_1 = PayoffMatrix1[a_1, a_2]
    r_2 = PayoffMatrix2[a_1, a_2]

    if i > n_episodes*0.8:
        av_rew += [r_1, r_2]
    
    # SINGLE STEP UPDATE    
    Player1.single_step_update(s, a_1, r_1, s, done)
    Player2.single_step_update(s, a_2, r_2, s, done)        

    traj_policy_1 = np.append(traj_policy_1, Player1.get_policy([0])[0])
    traj_policy_2 = np.append(traj_policy_2, Player2.get_policy([0])[0])
      
print(av_rew/n_episodes)

np.savetxt('FirstTry.txt', np.append(traj_policy_1.reshape(-1,1), traj_policy_2.reshape(-1,1), axis=1))

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 [-0.05987  0.05987]


### Replicator dynamics

This kind of evolution of probabilities is known in Evolutionary Game Theory as Replicator Dynamics. 

Replicator Dynamics is a mathematical model of growth that is used to study competing strategies/populations. In RD the population is divided into different types $x_i$, and each class has a *fitness* (how good are they) $f(x_i)$. The average *fitness* of the population is $\phi(x)$.


**In RD, the rate of growth of each class is proportional to how much better their indivual fitness is in respect to the average**.

$$
\dot{x_i} = x_i (f(x_i) - \phi(x))
$$

In our case the types of population are the different actions and their probability. The fitness is how well they do against the opponent!

$$
x_i \rightarrow \pi^{(1)}(i) \\
f(x_i) \rightarrow \sum_k A_{i k} \pi^{(2)}(k)
$$
The average fitness is just:

$$
\phi(x) = \sum_{j, k} A_{j k} \pi^{(1)}(j) \pi^{(2)}(k)
$$

So we have that doing Multi Agent Reinforcement Learning with Natural Gradient in Matrix Games is equivalent to let the policies "evolve" with the Replicator Dynamics. We don't really need an algorithm, and can directly solve the differential equations for the policies.



In [4]:
# The solution for REPLICATOR DYNAMICS

from scipy.integrate import solve_ivp as solve

p1_0 = 0.65
p2_0 = 0.65

# Payoff matrix for player 1
A = np.array([[-1, 1],
              [1, -1]])

# Payoff matrix for player 2
B = np.array([[1, -1],
              [-1, 1]])
             
    
# Scipy want a function   fun(t, p) to integrate numerically:
# d(p)/dt = fun(t, p)
def wrapper_payoff(A, B):
    def payoff(t, p):
        
        p1 = np.array([p[0], 1-p[0]])
        p2 = np.array([p[1], 1-p[1]])
        
        # Matrix multiplication between A and pi^(2)
        # sum_j A_1j p^(2)(j) = FITNESS OF ACTION 1 for player 1
        # sum_j A_2j p^(2)(j) = FITNESS OF ACTION 2 for player 1
        A_x_p2 = np.matmul(A, p2)
        # Matrix multiplication between A and pi^(1)
        # sum_j B_j1 p^(1)(j) = FITNESS OF ACTION 1 for player 2
        # sum_j B_j1 p^(1)(j) = FITNESS OF ACTION 2 for player 2
        B_x_p1 = np.matmul(B, p1)
        
        # Average fitness for player 1
        p1T_A_x_p2 = np.dot(p1, A_x_p2)
        # Average fitness for player 2                   
        p2T_B_x_p1 = np.dot(p2, B_x_p1)

        expected_payoff = np.zeros(2)
                   
        # REPLICATOR FORMULA for differential update!
        expected_payoff[0] = p1[0] * (A_x_p2[0]  - p1T_A_x_p2)
        expected_payoff[1] = p2[0] * (B_x_p1[0]  - p2T_B_x_p1)
        return expected_payoff
    # this returns the above function.
    return payoff

sol = solve(wrapper_payoff(A,np.transpose(B)), (0,100), np.array([p1_0, p2_0]),first_step=0.01, max_step=0.05)
np.savetxt('ODE_solution.txt', np.transpose(sol.y))

(
### Social Dilemmas

Prisoner's Dilemma: https://en.wikipedia.org/wiki/Prisoner%27s_dilemma 

Two possibilities: Accuse other / Do not accuse other.

$$
\text{Years in Prison} = \begin{bmatrix} 
-6, -6 & 0, -7 \\
-7, 0 & -1, -1\\
\end{bmatrix} 
$$

)

### Stochastic reward

So far we have dealt with deterministic payoffs: Given the two actions $a_1$ and $a_2$ the results is perfectly determined. Note that this does not need to be true, we could also have that *on average* the payoffs could be that of the matrix, but each single game could have a stochastic payoff. 


While for sure an human would be much confused (try play football if sometimes you have to score in one goal and sometimes in the other and you never know where...) the algorithm works even in that case!
